In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

train_df = pd.read_table('../input/train.tsv')
test_df = pd.read_table('../input/test.tsv')
drop_list = ['car name', 'horsepower', 'id']
train_df.drop(drop_list, axis=1, inplace=True)
test_id = test_df.id.values
test_df.drop(drop_list, axis=1, inplace=True)
feature_list = ['weight', 'model year']
print("train data shape:{}, test data shape:{}".format(train_df.shape, test_df.shape))

corr = train_df.corr()
sns.heatmap(corr, square=True, annot=True);

Using TensorFlow backend.


train data shape:(199, 7), test data shape:(199, 6)


In [11]:
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

N = 5
kf = KFold(n_splits=N)

def create_model(params, train_data_set, test_data_set):
    return lgb.train(params,                                                                                  
                    train_data_set,                                                                               
                    num_boost_round=200,                                                                      
                    valid_sets=test_data_set,                                                                     
                    early_stopping_rounds=10,
                    verbose_eval=False
                    )

def objective(trial):
    #最適化するパラメータの設定
    num_leaves = trial.suggest_int("num_leaves", 7, 40)
    learning_rate = trial.suggest_uniform("learning_rate", 0.001, 0.1)
    feature_fraction = trial.suggest_uniform("feature_fraction", 0, 1)
    bagging_fraction = trial.suggest_uniform("bagging_fraction", 0, 1)
    bagging_freq = trial.suggest_int("bagging_freq", 0, 10)
    lambda_l2 = trial.suggest_uniform("lambda_l2", 0, 3)
    params = {                                                                                               
        'boosting_type': 'gbdt',                                                                             
        'objective': 'regression_l2',                                                        
        'metric': 'RMSE',                                               
        'num_leaves': num_leaves,                                                      
        'learning_rate': learning_rate,                                                         
        'feature_fraction': feature_fraction,                                                                  
        'bagging_fraction': bagging_fraction,                                                                       
        'bagging_freq': bagging_freq,
        'lambda_l2': lambda_l2,
    }

    test_pred = np.zeros(len(test_df))
    accuracy_train = []
    index = 1
    for train, test in kf.split(train_df):
        X_train = train_df.iloc[train]
        y_train = X_train['mpg'].values
        X_train = X_train[feature_list].values

        X_test = train_df.iloc[test]
        y_test = X_test['mpg'].values
        X_test = X_test[feature_list].values

        train_data_set = lgb.Dataset(X_train, y_train)
        test_data_set = lgb.Dataset(X_test, y_test, reference=train_data_set)

        model = create_model(params, train_data_set, test_data_set)
        pred_train = model.predict(X_test, num_iteration=model.best_iteration)

        test_data = test_df[feature_list].values
        test_pred += model.predict(test_data, num_iteration=model.best_iteration)/N
        accuracy_train.append(np.sqrt(mean_squared_error(y_test, pred_train)))
         # print('Fold {} CV RMSE: {:.6}'.format(index, accuracy_train[-1]))
        index += 1
    ave = sum(accuracy_train)/len(accuracy_train)
    print('CV average RMSE: {:.6}'.format(ave))

    #検証用データに対する正答率が最大となるハイパーパラメータを求める
    return ave

In [13]:
study = optuna.create_study()
study.optimize(objective, n_trials=1000)

CV average RMSE: 7.77498


[I 2019-03-13 04:49:58,095] Finished a trial resulted in value: 7.774984694620786. Current best value is 7.774984694620786 with parameters: {'num_leaves': 34, 'learning_rate': 0.0595267732315347, 'feature_fraction': 0.8173408591452026, 'bagging_fraction': 0.08308789606393119, 'bagging_freq': 9, 'lambda_l2': 2.614351559439201}.


CV average RMSE: 7.77498


[I 2019-03-13 04:49:58,153] Finished a trial resulted in value: 7.774984694620786. Current best value is 7.774984694620786 with parameters: {'num_leaves': 34, 'learning_rate': 0.0595267732315347, 'feature_fraction': 0.8173408591452026, 'bagging_fraction': 0.08308789606393119, 'bagging_freq': 9, 'lambda_l2': 2.614351559439201}.


CV average RMSE: 2.96874


...

[I 2019-03-13 04:56:36,241] Finished a trial resulted in value: 2.8364217971586205. Current best value is 2.7896716317660784 with parameters: {'num_leaves': 36, 'learning_rate': 0.09976132913222707, 'feature_fraction': 0.07700457447759154, 'bagging_fraction': 0.9045363739640161, 'bagging_freq': 7, 'lambda_l2': 1.1318936905174553}.


CV average RMSE: 2.84832


[I 2019-03-13 04:56:36,793] Finished a trial resulted in value: 2.8483233937470898. Current best value is 2.7896716317660784 with parameters: {'num_leaves': 36, 'learning_rate': 0.09976132913222707, 'feature_fraction': 0.07700457447759154, 'bagging_fraction': 0.9045363739640161, 'bagging_freq': 7, 'lambda_l2': 1.1318936905174553}.


CV average RMSE: 2.855


[I 2019-03-13 04:56:37,323] Finished a trial resulted in value: 2.854997640080156. Current best value is 2.7896716317660784 with parameters: {'num_leaves': 36, 'learning_rate': 0.09976132913222707, 'feature_fraction': 0.07700457447759154, 'bagging_fraction': 0.9045363739640161, 'bagging_freq': 7, 'lambda_l2': 1.1318936905174553}.


In [14]:
study.best_value

2.7896716317660784

In [15]:
study.best_params

{'num_leaves': 36,
 'learning_rate': 0.09976132913222707,
 'feature_fraction': 0.07700457447759154,
 'bagging_fraction': 0.9045363739640161,
 'bagging_freq': 7,
 'lambda_l2': 1.1318936905174553}

In [18]:
params = {                                                                                   
        'boosting_type': 'gbdt',                                                                             
        'objective': 'regression_l2',                                                        
        'metric': 'RMSE',                                               
        'num_leaves': study.best_params['num_leaves'],                                                      
        'learning_rate': study.best_params['learning_rate'],                                                         
        'feature_fraction': study.best_params['feature_fraction'],                                                                  
        'bagging_fraction': study.best_params['bagging_fraction'],                                                                       
        'bagging_freq': study.best_params['bagging_freq'],
        'lambda_l2': study.best_params['lambda_l2'],
    }

In [21]:
test_pred = np.zeros(len(test_df))
accuracy_train = []
index = 1
for train, test in kf.split(train_df):
    X_train = train_df.iloc[train]
    y_train = X_train['mpg'].values
    X_train = X_train[feature_list].values

    X_test = train_df.iloc[test]
    y_test = X_test['mpg'].values
    X_test = X_test[feature_list].values

    train_data_set = lgb.Dataset(X_train, y_train)
    test_data_set = lgb.Dataset(X_test, y_test, reference=train_data_set)

    model = create_model(params, train_data_set, test_data_set)
    pred_train = model.predict(X_test, num_iteration=model.best_iteration)

    test_data = test_df[feature_list].values
    test_pred += model.predict(test_data, num_iteration=model.best_iteration)/N
    accuracy_train.append(np.sqrt(mean_squared_error(y_test, pred_train)))
    print('Fold {} CV RMSE: {:.6}'.format(index, accuracy_train[-1]))
    index += 1
ave = sum(accuracy_train)/len(accuracy_train)

Fold 1 CV RMSE: 2.90328
Fold 2 CV RMSE: 2.94539
Fold 3 CV RMSE: 2.47432
Fold 4 CV RMSE: 3.09358
Fold 5 CV RMSE: 2.53179


In [22]:
import csv
f = open('lgbm.csv', 'w')
writer = csv.writer(f, lineterminator='\n')
csvlist = [[] for i in range(len(test_pred))]
for i in range(len(test_pred)):
    csvlist[i].append(test_id[i])
    csvlist[i].append(test_pred[i])
    writer.writerow(csvlist[i])
f.close()